In [1]:
import glob
import os
import sys
import time
import random

import carla

import torch
from torch.utils.data import Dataset

import carla
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

from PIL import Image

from agents.navigation.basic_agent import BasicAgent

import gym
import gym_carla

import torch.nn as nn
from itertools import count
import torch.nn.functional as F

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# parameters for the gym_carla environment
params = {
    'number_of_vehicles': 0,
    'number_of_walkers': 0,
    'display_size': 256,  # screen size of bird-eye render
    'max_past_step': 1,  # the number of past steps to draw
    'dt': 0.1,  # time interval between two frames
    'discrete': True,  # whether to use discrete control space
    'discrete_acc': [2.0],  # discrete value of accelerations
    'discrete_steer': [-0.3, 0.0, 0.3],  # discrete value of steering angles
    'continuous_accel_range': [-3.0, 3.0],  # continuous acceleration range
    'continuous_steer_range': [-0.3, 0.3],  # continuous steering angle range
    'ego_vehicle_filter': 'vehicle.lincoln*',  # filter for defining ego vehicle
    'port': 2000,  # connection port
    'town': 'Town04',  # which town to simulate
    'task_mode': 'random',  # mode of the task, [random, roundabout (only for Town03)]
    'max_time_episode': 500,  # maximum timesteps per episode
    'max_waypt': 12,  # maximum number of waypoints
    'obs_range': 32,  # observation range (meter)
    'lidar_bin': 0.125,  # bin size of lidar sensor (meter)
    'd_behind': 12,  # distance behind the ego vehicle (meter)
    'out_lane_thres': 1.8,  # threshold for out of lane
    'desired_speed': 5,  # desired speed (m/s)
    'max_ego_spawn_times': 20,  # maximum times to spawn ego vehicle
    'display_route': True,  # whether to render the desired route
    'pixor_size': 64,  # size of the pixor labels
    'pixor': False,  # whether to output PIXOR observation
    'use_fixed':False,
    'weather':'ClearNoon',
    'Collect_Data':True
}

# Set gym-carla environment
env = gym.make('carla-v0', params=params)

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
connecting to Carla server...
Carla server connected!
WeatherParameters(cloudiness=5.000000, cloudiness=5.000000, precipitation=0.000000, precipitation_deposits=0.000000, wind_intensity=10.000000, sun_azimuth_angle=-1.000000, sun_altitude_angle=45.000000, fog_density=2.000000, fog_distance=0.750000, fog_falloff=0.100000, wetness=0.000000, scattering_intensity=1.000000, mie_scattering_scale=0.030000, rayleigh_scattering_scale=0.033100)
372


In [3]:
class DQN(nn.Module):

    def __init__(self, outputs):
        super(DQN, self).__init__()
        
        self.lin1 = nn.Linear(9,80)
        self.lin2 = nn.Linear(80,50)
        self.lin3 = nn.Linear(50,25)
        self.lin4 = nn.Linear(25,15)
        self.lin5 = nn.Linear(15,8)
        self.lin6 = nn.Linear(8,3)

    def forward(self, x):
        x = x.to(device)
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = F.relu(self.lin3(x))
        x = F.relu(self.lin4(x))
        x = F.relu(self.lin5(x))
        x = self.lin6(x)
        
        return x.view(x.size(0), -1)

In [4]:
vis_net = DQN(3).to(device)
vis_net.load_state_dict(torch.load('./model_params_CL/model_10.final'))
vis_net.eval()

DQN(
  (lin1): Linear(in_features=9, out_features=80, bias=True)
  (lin2): Linear(in_features=80, out_features=50, bias=True)
  (lin3): Linear(in_features=50, out_features=25, bias=True)
  (lin4): Linear(in_features=25, out_features=15, bias=True)
  (lin5): Linear(in_features=15, out_features=8, bias=True)
  (lin6): Linear(in_features=8, out_features=3, bias=True)
)

In [9]:
num_episodes = 20
min_overall_loss = 1000
data = DataCollector(env)
for i_episode in range(num_episodes):
    rewards = 0
    # Initialize the environment and state
    obs = env.reset()
    #ego_dir retirves the distance and angle from vehicle to nearest waypoint
    ego_location = env.ego.get_location()
    ego_dir = gym_carla.envs.misc.get_lane_dis(env.waypoints,ego_location.x,ego_location.y)
    #pos gets a distanc d and array w which has to be seperated out in below line
    ego_pos = np.asarray((ego_dir[0],ego_dir[1][0],ego_dir[1][1]),dtype=np.float32)
    state = np.concatenate((ego_pos,np.zeros(6)))
    
    #inital state with previous action for data collection consistency
    action = np.array([0])
    reward = 0
    
    data.collect_step(obs, state, reward, action)
    
    state = torch.tensor(state)
    loss = episode_loss = 1000
    for t in count():
        # Select and perform an action
        with torch.no_grad():
            action = vis_net(state.float()).argmax().view(1,1)
        next_obs, reward, done, info  = env.step(random.randint(0,2))
        rewards += reward
    
        #pos gets a distanc d and array w which has to be seperated out in below line
        pos = np.asarray((info['position'][0],info['position'][1][0],info['position'][1][1]))
        ang = np.asarray(info['angular_vel'])
        acc = np.asarray(info['acceleration'])
        steer = np.asarray(info['steer'])
        next_state = np.concatenate((pos, ang, acc, steer), axis=None)
        
        #collect data
        
        data.collect_step(obs,state,reward,np.array([action.item()]))
        
        #update state
        state = torch.tensor(next_state)
        obs = next_obs
        #obs = torch.tensor(next_obs)
        
        
        if done:
            print('########')
            data.save_episode()
            break

########
########
########
########
########
########
########
########
########
########
########
########
########
########
########
########
########
########
########
########


In [12]:
image = obs['semantic_img']
image

array([[[ 70, 130, 180],
        [ 70, 130, 180],
        [ 70, 130, 180],
        ...,
        [ 70, 130, 180],
        [ 70, 130, 180],
        [ 70, 130, 180]],

       [[ 70, 130, 180],
        [ 70, 130, 180],
        [ 70, 130, 180],
        ...,
        [ 70, 130, 180],
        [ 70, 130, 180],
        [ 70, 130, 180]],

       [[ 70, 130, 180],
        [ 70, 130, 180],
        [ 70, 130, 180],
        ...,
        [ 70, 130, 180],
        [ 70, 130, 180],
        [ 70, 130, 180]],

       ...,

       [[128,  64, 128],
        [128,  64, 128],
        [128,  64, 128],
        ...,
        [128,  64, 128],
        [128,  64, 128],
        [128,  64, 128]],

       [[128,  64, 128],
        [128,  64, 128],
        [128,  64, 128],
        ...,
        [128,  64, 128],
        [128,  64, 128],
        [128,  64, 128]],

       [[128,  64, 128],
        [  0,   0, 142],
        [  0,   0, 142],
        ...,
        [  0,   0, 142],
        [128,  64, 128],
        [128,  64, 128]]

In [21]:
carla.ColorConverter.CityScapesPalette

carla.libcarla.ColorConverter.CityScapesPalette

In [9]:
obs['state']

array([-1.34325527e-06,  1.74249798e-07,  4.39107120e-08,  0.00000000e+00])

In [ ]:
import os
import pandas as pd
from torchvision.io import read_image

class CustomImageDataset(Dataset):
    def __init__(self, weather, town, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        dir_str = './Datasets/'+weather+'/'+town
        self.RBG_dir = dir_str+'/RGB'
        self.sem_dir = dir_str+'/Semantic'
        self.states = dir_str+'/States'
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(os.listdir(self.states))

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [5]:
import time
import os
class DataCollector():
    def __init__(self, env, recorder = False):
        self.timestamp = time.strftime("%a, %d %b %Y %H:%M:%S")
        if not env:
            raise Exception("Please provide a carla gym env object")
        self.rgb_dir = './Datasets/'+env.weather+'/'+env.town+'/RGB/'
        self.sem_dir = './Datasets/'+env.weather+'/'+env.town+'/Semantic/'
        self.state_dir = './Datasets/'+env.weather+'/'+env.town+'/States/'
        self.recording_dir = './Datasets/'+env.weather+'/'+env.town+'/Recordings/'
        self.eps_rgb = []
        self.eps_sem = []
        self.eps_state = []
        self.recorder = recorder
        
        if self.recorder:
            self.record()
    
    def reset(self):
        #reset lists and time
        self.eps_rgb = []
        self.eps_sem = []
        self.eps_state = []
        
        self.timestamp = time.strftime("%a, %d %b %Y %H:%M:%S")
        
        #stop and restart recorder
        if self.recorder:
            env.client.stop_recorder()
            self.record()
            
            
    
    def collect_step(self, obs, state, reward, action):
        # save image data
        self.eps_rgb.append(obs['camera'])
        self.eps_sem.append(obs['semantic'])
        #convert tensor to numpy array and append action to state vector
        if torch.is_tensor(state):
            state = state.cpu()
            state = state.numpy()
        state = np.append(state,action)
        self.eps_state.append(state)
        
        
    def save_episode(self):
        
        #save data for episode in numpy arrays
        np.savez(self.rgb_dir+self.timestamp, *self.eps_rgb)
        np.savez(self.sem_dir+self.timestamp, *self.eps_sem)
        np.savez(self.state_dir+self.timestamp, *self.eps_state)
        
        self.reset()
        
    def record(self):
        env.client.start_recorder(self.recording_dir+self.timestamp+".log")
        

In [9]:
x = np.load('./Datasets/'+env.weather+'/'+env.town+'/Semantic/Tue, 25 Jan 2022 10:06:25.npz')

In [16]:
x['arr_3']

array([[[13,  0,  0],
        [13,  0,  0],
        [13,  0,  0],
        ...,
        [13,  0,  0],
        [13,  0,  0],
        [13,  0,  0]],

       [[13,  0,  0],
        [13,  0,  0],
        [13,  0,  0],
        ...,
        [13,  0,  0],
        [13,  0,  0],
        [13,  0,  0]],

       [[13,  0,  0],
        [13,  0,  0],
        [13,  0,  0],
        ...,
        [13,  0,  0],
        [13,  0,  0],
        [13,  0,  0]],

       ...,

       [[10,  0,  0],
        [10,  0,  0],
        [10,  0,  0],
        ...,
        [10,  0,  0],
        [10,  0,  0],
        [10,  0,  0]],

       [[10,  0,  0],
        [10,  0,  0],
        [10,  0,  0],
        ...,
        [10,  0,  0],
        [10,  0,  0],
        [10,  0,  0]],

       [[10,  0,  0],
        [10,  0,  0],
        [10,  0,  0],
        ...,
        [10,  0,  0],
        [10,  0,  0],
        [10,  0,  0]]], dtype=uint8)

In [11]:
im = Image.fromarray(x['arr_30'],'RGB')

In [12]:
im.show()

In [ ]:
env.client.start_recorder('./Datasets/'+env.weather+'/'+env.town+'/Recordings/test.log')

In [5]:
env.client.stop_recorder()

NameError: name 'env' is not defined

In [19]:
env.client.start_recorder('~/')

''

In [7]:
x = carla.ColorConverter.CityScapesPalette

In [14]:
x = np.random.rand(10,10,4)
x

array([[[0.4321363 , 0.61397539, 0.67983316, 0.78845075],
        [0.76070553, 0.2097094 , 0.92695049, 0.88333433],
        [0.80511614, 0.5831015 , 0.83937924, 0.02730096],
        [0.63369737, 0.02548763, 0.09131613, 0.40046942],
        [0.75732817, 0.38368949, 0.47868235, 0.58427476],
        [0.95555036, 0.64882456, 0.2510238 , 0.22522981],
        [0.38106515, 0.50081385, 0.02176914, 0.28932587],
        [0.64581972, 0.3895502 , 0.75850214, 0.79847408],
        [0.75315637, 0.15394866, 0.31907138, 0.35450717],
        [0.73739602, 0.15554188, 0.33031822, 0.61732198]],

       [[0.0343237 , 0.46469612, 0.17527633, 0.74191989],
        [0.38140428, 0.31879894, 0.79703187, 0.30351461],
        [0.1803821 , 0.27746248, 0.2613625 , 0.29108117],
        [0.52661891, 0.16746154, 0.18409258, 0.60957093],
        [0.05353854, 0.18848794, 0.68059866, 0.8369277 ],
        [0.52230812, 0.37359954, 0.23082339, 0.96170113],
        [0.57455868, 0.22770107, 0.71782914, 0.15190779],
        [0.1

In [19]:
y = x[:,:,:3]
y

array([[[0.4321363 , 0.61397539, 0.67983316],
        [0.76070553, 0.2097094 , 0.92695049],
        [0.80511614, 0.5831015 , 0.83937924],
        [0.63369737, 0.02548763, 0.09131613],
        [0.75732817, 0.38368949, 0.47868235],
        [0.95555036, 0.64882456, 0.2510238 ],
        [0.38106515, 0.50081385, 0.02176914],
        [0.64581972, 0.3895502 , 0.75850214],
        [0.75315637, 0.15394866, 0.31907138],
        [0.73739602, 0.15554188, 0.33031822]],

       [[0.0343237 , 0.46469612, 0.17527633],
        [0.38140428, 0.31879894, 0.79703187],
        [0.1803821 , 0.27746248, 0.2613625 ],
        [0.52661891, 0.16746154, 0.18409258],
        [0.05353854, 0.18848794, 0.68059866],
        [0.52230812, 0.37359954, 0.23082339],
        [0.57455868, 0.22770107, 0.71782914],
        [0.14378208, 0.33087038, 0.899426  ],
        [0.91195303, 0.98714188, 0.22466026],
        [0.31786438, 0.24190379, 0.85018805]],

       [[0.81797504, 0.36422064, 0.88699252],
        [0.11975938, 0.4314713

In [20]:
y[:,:,::-1]

array([[[0.67983316, 0.61397539, 0.4321363 ],
        [0.92695049, 0.2097094 , 0.76070553],
        [0.83937924, 0.5831015 , 0.80511614],
        [0.09131613, 0.02548763, 0.63369737],
        [0.47868235, 0.38368949, 0.75732817],
        [0.2510238 , 0.64882456, 0.95555036],
        [0.02176914, 0.50081385, 0.38106515],
        [0.75850214, 0.3895502 , 0.64581972],
        [0.31907138, 0.15394866, 0.75315637],
        [0.33031822, 0.15554188, 0.73739602]],

       [[0.17527633, 0.46469612, 0.0343237 ],
        [0.79703187, 0.31879894, 0.38140428],
        [0.2613625 , 0.27746248, 0.1803821 ],
        [0.18409258, 0.16746154, 0.52661891],
        [0.68059866, 0.18848794, 0.05353854],
        [0.23082339, 0.37359954, 0.52230812],
        [0.71782914, 0.22770107, 0.57455868],
        [0.899426  , 0.33087038, 0.14378208],
        [0.22466026, 0.98714188, 0.91195303],
        [0.85018805, 0.24190379, 0.31786438]],

       [[0.88699252, 0.36422064, 0.81797504],
        [0.31479602, 0.4314713